In [1]:
from functools import *
from itertools import *

import pandas as pd
import torch
import torch.nn.functional as F
from transformers import *

chaini = chain.from_iterable

In [2]:
device = torch.device('cuda:0')
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.bos_token
lm = GPT2LMHeadModel.from_pretrained('gpt2').eval().to(device)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /home/yugai/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_lab

In [3]:
# query = 'Permission to make digital or hard copies of all or part of this work for personal or'
# cands = [
#     'classroom use is granted without fee provided that copies are not made or distributed for profit or commercial advantage and that copies bear this notice and the full citation on the first page. To copy otherwise, to republish, to post on servers or to redistribute to lists, requires prior specific permission and/or a fee. PLDI’09, June 15–20, 2009, Dublin, Ireland.',
#     'and is used for the application logic of browser-based productivity applications such as Google Mail, Google Docs and Zimbra Collaboration Suite. In this domain, in order to provide a fluid user experience and enable a new generation of applications, virtual machines must provide a low startup time and high performance.',
#     'We present a trace-based compilation technique for dynamic languages that reconciles speed of compilation with excellent performance of the generated machine code. Our system uses a mixedmode execution approach: the system starts running JavaScript in a fast-starting bytecode interpreter. As the program runs, the system identifies hot (frequently executed) bytecode sequences, records them, and compiles them to fast native code. We call such a sequence of instructions a trace. Unlike method-based dynamic compilers, our dynamic compiler operates at the granularity of individual loops. This design choice is based on the expectation that programs spend most of their time in hot loops. Even in dynamically typed languages, we expect hot loops to be mostly type-stable, meaning that the types of values are invariant. (12) For example, we would expect loop counters that start as integers to remain integers for all iterations. When both of these expectations hold, a trace-based compiler can cover the program execution with a small number of type-specialized, efficiently compiled traces.'
# ]

query = 'Dynamic languages such as JavaScript, Python, and Ruby, are popular since they are expressive, accessible to non-experts, and make deployment as easy as distributing a source file. They are used for small scripts as well as for complex applications. JavaScript, for example, is the de facto standard for client-side web programming'
cands = [
    'Permission to make digital or hard copies of all or part of this work for personal or',
    'classroom use is granted without fee provided that copies are not made or distributed for profit or commercial advantage and that copies bear this notice and the full citation on the first page. To copy otherwise, to republish, to post on servers or to redistribute to lists, requires prior specific permission and/or a fee. PLDI’09, June 15–20, 2009, Dublin, Ireland.',
    'Copyright c © 2009 ACM 978-1-60558-392-1/09/06... $5.00',
    'and is used for the application logic of browser-based productivity applications such as Google Mail, Google Docs and Zimbra Collaboration Suite. In this domain, in order to provide a fluid user experience and enable a new generation of applications, virtual machines must provide a low startup time and high performance.',
    'Compilers for statically typed languages rely on type information to generate efficient machine code. In a dynamically typed programming language such as JavaScript, the types of expressions may vary at runtime. This means that the compiler can no longer easily transform operations into machine instructions that operate on one specific type. Without exact type information, the compiler must emit slower generalized machine code that can deal with all potential type combinations. While compile-time static type inference might be able to gather type information to generate optimized machine code, traditional static analysis is very expensive and hence not well suited for the highly interactive environment of a web browser.'
]

In [4]:
def forward(text, tokenizer, lm):
    inputs = tokenizer.batch_encode_plus(text,
                                         padding=True,
                                         truncation=True,
                                         return_tensors='pt',
                                         return_token_type_ids=True)
    with torch.no_grad():
        outputs = lm(**inputs.to(device))
        input_ids = F.pad(inputs['input_ids'], [-1, 1, 0, 0],
                          value=tokenizer.eos_token_id)[:, :, None]
        log_probs = outputs['logits'].log_softmax(2).gather(
            2, input_ids).squeeze(2)
    return inputs, outputs, input_ids, log_probs


def display_as_df(input_ids, log_probs):
    df = pd.DataFrame(
        dict(
            chaini([[
                f'input_ids[{i}]',
                tokenizer.convert_ids_to_tokens(input_ids[i])
            ], [f'log_probs[{i}]', [round(_.item(), 3) for _ in log_probs[i]]]]
                   for i in range(len(input_ids)))))
    with pd.option_context('display.max_rows', len(df) + 1):
        display(df)

In [5]:
inputs, outputs, input_ids, log_probs = forward(
    [f'{tokenizer.eos_token} {query} {cand}' for cand in cands], tokenizer, lm)
display_as_df(input_ids, log_probs)

,input_ids[0],log_probs[0],input_ids[1],log_probs[1],input_ids[2],log_probs[2],input_ids[3],log_probs[3],input_ids[4],log_probs[4]
0,ĠDynamic,-13.726,ĠDynamic,-13.726,ĠDynamic,-13.726,ĠDynamic,-13.726,ĠDynamic,-13.726
1,Ġlanguages,-12.636,Ġlanguages,-12.636,Ġlanguages,-12.636,Ġlanguages,-12.636,Ġlanguages,-12.636
2,Ġsuch,-11.416,Ġsuch,-11.416,Ġsuch,-11.416,Ġsuch,-11.416,Ġsuch,-11.416
3,Ġas,-8.032,Ġas,-8.032,Ġas,-8.032,Ġas,-8.032,Ġas,-8.032
4,ĠJavaScript,-9.216,ĠJavaScript,-9.216,ĠJavaScript,-9.216,ĠJavaScript,-9.216,ĠJavaScript,-9.216
5,",",-6.665,",",-6.665,",",-6.665,",",-6.665,",",-6.665
6,ĠPython,-9.786,ĠPython,-9.786,ĠPython,-9.786,ĠPython,-9.786,ĠPython,-9.786
7,",",-5.491,",",-5.491,",",-5.491,",",-5.491,",",-5.491
8,Ġand,-5.694,Ġand,-5.694,Ġand,-5.694,Ġand,-5.694,Ġand,-5.694
9,ĠRuby,-10.220,ĠRuby,-10.220,ĠRuby,-10.220,ĠRuby,-10.220,ĠRuby,-10.220


In [6]:
inputs, outputs, input_ids, log_probs = forward(
    [f'{tokenizer.eos_token} {cand}' for cand in cands], tokenizer, lm)
display_as_df(input_ids, log_probs)

,input_ids[0],log_probs[0],input_ids[1],log_probs[1],input_ids[2],log_probs[2],input_ids[3],log_probs[3],input_ids[4],log_probs[4]
0,ĠPer,-12.003,Ġclassroom,-16.148,ĠCopyright,-11.774,Ġand,-7.321,ĠComp,-12.126
1,mission,-2.304,Ġuse,-9.842,Ġc,-9.179,Ġis,-8.200,ilers,-11.297
2,Ġto,-5.689,Ġis,-7.257,ĠÂ©,-8.204,Ġused,-9.625,Ġfor,-7.418
3,Ġmake,-6.014,Ġgranted,-12.065,Ġ2009,-7.932,Ġfor,-7.518,Ġstatically,-14.316
4,Ġdigital,-10.304,Ġwithout,-10.048,ĠAC,-10.713,Ġthe,-5.082,Ġtyped,-7.515
5,Ġor,-7.648,Ġfee,-13.409,M,-8.827,Ġapplication,-12.459,Ġlanguages,-9.845
6,Ġhard,-11.497,Ġprovided,-12.053,Ġ978,-12.914,Ġlogic,-12.393,Ġrely,-11.713
7,Ġcopies,-12.340,Ġthat,-9.280,-,-5.049,Ġof,-9.289,Ġon,-8.522
8,Ġof,-3.697,Ġcopies,-12.823,1,-7.165,Ġbrowser,-12.568,Ġtype,-7.681
9,Ġall,-6.008,Ġare,-6.170,-,-3.084,-,-6.905,Ġinformation,-11.330
